<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 878.7 kB/s eta 0:00:00


In [12]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp

from scipy.optimize import root_scalar

In [24]:
p=Props(['Benzene','Toluene'])
z = jnp.array([0.6,0.4])

In [7]:
def bubblePy_ideal(x, T):
    Pi = x*p.Pvap(T)
    bubbleP = np.sum(Pi)
    return bubbleP, Pi/bubbleP

In [42]:
@jax.jit
def bubbleTy_ideal_eq(x, T, P):
    return jnp.sum(x*p.Pvap(T)/P) - 1.

bubbleTy_ideal_eq_gradT = jax.jit(jax.grad(bubbleTy_ideal_eq, 1))
bubbleTy_ideal_eq_grad2T = jax.jit(jax.grad(bubbleTy_ideal_eq_gradT, 1))



def bubbleTy_ideal(x, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(x*Tb)

    res = root_scalar(lambda T: bubbleTy_ideal_eq(x,T,P), x0=Tguess,
                fprime=lambda T: bubbleTy_ideal_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_ideal_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    return res.root

def dewPx_ideal(y, T):
    y_div_Psat = y/p.Pvap(T)
    dewP = 1./jnp.sum(y_div_Psat)
    return dewP, y_div_Psat*dewP

@jax.jit
def dewTx_ideal_eq(y, T, P):
    return jnp.sum(y*P/p.Pvap(T)) - 1.

dewTx_ideal_eq_gradT = jax.jit(jax.grad(dewTx_ideal_eq, 1))
dewTx_ideal_eq_grad2T = jax.jit(jax.grad(dewTx_ideal_eq_gradT, 1))

def dewTx_ideal(y, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(y*Tb)
    res = root_scalar(lambda T: dewTx_ideal_eq(y,T,P), x0=Tguess,
                fprime=lambda T: dewTx_ideal_eq_gradT(y,T,P),
                fprime2=lambda T: dewTx_ideal_eq_grad2T(y,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    return res.root

In [43]:
dewTx_ideal(z, 2e5)

Array(393.56318087, dtype=float64)